In [4]:
from selenium.webdriver.common.by import By
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter
from openpyxl import Workbook
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
import time
import datetime
import requests
from selenium.webdriver.chrome.service import Service
import pickle
from update import update, last_addr
from tqdm import tqdm

# 파일 불러오기
with open('data/addr.pkl', 'rb') as f:
    name = pickle.load(f)

# 전에 어디까지 했는지 인덱스 번호
def make_query():
    last_str = last_addr()
    idx = name.index(last_str)
    return idx

def webdriver_setting():
    # Webdriver headless mode setting
    options = webdriver.ChromeOptions()
    options.add_argument('window-size=1920x1080')
    options.add_argument("disable-gpu")
    return options

# 플레이스 url 추출
def place_url(url, options):
    # Start crawling/scraping! py 파일로 빼야함
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    res = driver.get(url)
    driver.implicitly_wait(30)

    # 가게 클릭
    driver.find_element(By.XPATH, '//*[@id="root"]/div/div[2]/div/div/div[3]/div/div/div/div/div/div/div/div[3]/div[2]/ul/li/button').click()

    # 플레이스 링크 추출
    real_url = driver.find_element(By.XPATH, '//*[@id="root"]/div/div[2]/div/div/div[3]/div/div/div/iframe').get_attribute("src")
    print(real_url)
    driver.quit()

    return real_url

# 리뷰 스크래핑
def scraping(real_url, options):
    # Start crawling/scraping!
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    res = driver.get(real_url)
    driver.implicitly_wait(30)

    # 리뷰 버튼 찾기
    def find_review():
        for review_num in range(1,10):
            review_str = driver.find_element(By.XPATH, f'//*[@id="app-root"]/div/div/div/div[4]/div/div/div/div/a[{review_num}]').text # 이게 리뷰면 실행하도록
            if review_str == '리뷰':
                print('review success')
                break
        return review_num

    review_num = find_review()
    driver.find_element(By.XPATH, f'//*[@id="app-root"]/div/div/div/div[4]/div/div/div/div/a[{review_num}]').click() # 리뷰 클릭

    # Pagedown
    driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)

    try:
        for i in range(9): # default : 9
            driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a').click()
            time.sleep(0.4)

    except Exception as e:
        print('finish')

    data = []

    try:
        for i in tqdm(range(1,51)): #리뷰 50개
            str_ = driver.find_element(By.XPATH, f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li[{i}]/div[5]/a').text
            print(str_) # 리뷰데이터
            data.append(str)
            print('-'*20)
    except:
        print('Error')

    driver.close()

    return data

# 딕셔너리 만들기 주소:리뷰
def make_dic(query, data):
    a = dict()
    a[query] = data
    return a

def data_append(dic):
    with open('data/review_data.pkl', 'rb') as f:
        a = pickle.load(f)
        a.append(dic)
        return a

def save_data(a):
    with open('data/review_data.pkl', 'wb') as f:
        pickle.dump(a, f)


options = webdriver_setting() #driver 옵션
base_url = 'https://map.naver.com'
idx = make_query() #어디까지 했는지 인덱스 뽑기
query = name[idx] #주소
update(query) #로그저장
url = base_url + '/p/search/' + query # 주소검색
real_url = place_url(url, options) #플레이스 url
data = scraping(real_url, options) #스크래핑
dic = make_dic(query, data) #딕셔너리 만들기
review_lst = data_append(dic) #리스트에 추가
save_data(review_lst) #리스트 저장

AttributeError: 'NoneType' object has no attribute 'click'